<a href="https://colab.research.google.com/github/cowanweks/colabcat/blob/master/colabcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget https://github.com/hashcat/hashcat/releases/download/v6.1.0/hashcat-6.1.0.7z
! 7z x hashcat-6.1.0.7z
! hashcat-6.1.0/hashcat.bin -b  # Run benchmark

In [ ]:
# Check the Hashcat Version to make sure it's installed correctly
!hashcat-6.1.0/hashcat.bin --version

In [ ]:
# WPA2 EAPOL Benchmark (change this to check the speed of whichever hash you're cracking)
!hashcat-6.1.0/hashcat.bin -w 4 -m 11300 --benchmark

In [3]:
# Check GPU Specs
!nvidia-smi

Thu Jun 19 13:32:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# Get hashes to crack
# Make a directory called 'dothashcat' in your Google Drive before this
# with the 'hashes' subdirectory for storing hashes
from google.colab import drive

# This will ask you to go to a link and get an authorization code
# to let the .hashcat folder get synced to your Google Drive
drive.mount('/content/drive')

# Make a symbolic link between Google Drive
# and the local .hashcat directory
!ls /content/drive/My\ Drive/dothashcat
!ln -s /content/drive/My\ Drive/dothashcat /root/.hashcat

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
hashes


In [ ]:
# Show wordlists
!ls /root/.hashcat/wordlists

In [ ]:
# Check that your hashes are accessible to Colab
!ls /root/.hashcat/hashes

In [ ]:
# Start pure brute force cracking with a session name
# !hashcat-6.1.0/hashcat.bin -m 2500 -a3 --session $SESSION /root/.hashcat/hashes/test.hccapx
# !hashcat-6.1.0/hashcat.bin -m 2500 /root/.hashcat/hashes/test.hccapx -a 3 -w 4 --session test-session

# Restore the session
# !hashcat-6.1.0/hashcat.bin --session $SESSION --restore

# Crack with a wordlist
!hashcat-6.1.0/hashcat.bin -a 0 -m 11300 -w 3 -S /root/.hashcat/hashes/hashes.txt /root/.hashcat/wordlists/wordlist.txt

In [ ]:
# creating new directory to store cap files
!mkdir cap_files
# uploading cap
from google.colab import files

uploaded = files.upload()

for f_name in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=f_name, length=len(uploaded[f_name])))

!mv {f_name} "/content/cap_files/{f_name}"

In [ ]:
import glob
import subprocess
import re
from tqdm import tqdm
is_cracked = False
wordlists = list()
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Common-Credentials/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Cracked-Hashes/milw0rm-dictionary.txt'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Default-Credentials/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Honeypot-Captures/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Leaked-Databases/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Malware/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/WiFi-WPA/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/Software/*'):
  wordlists.append(wordlist)
for wordlist in glob.glob('/root/wordlists/SecLists/Passwords/*.txt'):
  wordlists.append(wordlist)

In [ ]:
def check_if_cracked(command):
  output = subprocess.check_output([command + " --show"], shell=True).decode()
  if len(output) == 0:
    return False
  else:
    return True



In [ ]:
cap_file = f"/content/cap_files/{f_name} "
for wordlist in tqdm(wordlists):
  command = 'hashcat -m 2500 ' + cap_file + wordlist
  subprocess.call([command],shell=True)
  if check_if_cracked(command):
    is_cracked = True
  if is_cracked:
    print("Password is cracked!!")
    cracked_password = subprocess.check_output([command + " --show"], shell=True).decode()
    cracked_password = cracked_password.split(':')[-1]
    print(f"Password : {cracked_password}")
    print(f"wordlist used : {wordlist}")
    break
  else:
    continue

if is_cracked == False:
  print("Password was not found in SecLists")
else:
  pass